---
layout:     post
title:      "Inverted Pendulum Dynamics"
date:       2017-04-04 12:00:00
author:     Andrew
header-img: img/posts/sympy_dynamics/balance_rock.jpg
header-credit: https://unsplash.com/@nbmat
tags:       programming dynamics sympy python
---

So, let's get down to business.

The task of balancing an inverted pendulum is likely to require many different attempts to tune the control system so that it can perform decently.  Doing this physically would take quite a bit of time, so it makes sense to create a model for the pendulum system and simulate before moving to the physical system.

The double pendulum is a very thoroughly studied mechanical system for several reasons:
1. It's interesting.  the double pendulum is a simple example of chaotic motion
2. There's some interesting math behind it as well, and the equations of motion are not easily calculated
3.  Does all this while being fairly simple.

The first thing I did was to label all my variables:

![diagram]({{ site.baseurl }}/img/posts/sympy_dynamics/simple_double_diagram.png)

In this system, there are two degrees of freedom,the angles $$\theta_0$$ and $$\theta_1$$.  The state of the system can be described completely with these two angles.  What that means is that we're interested in calculating how those angles change over time.

In [1]:
from sympy import symbols, init_printing, S, Derivative, diff, simplify, solve, lambdify, cos, sin, expand, collect
from sympy.physics.vector import vlatex
from sympy.abc import a, b, c, d
import numpy as np
import scipy.integrate as integrate
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML, display, Math

rc('animation', html='html5')

init_printing(latex_printer=vlatex, latex_mode='equation')

In [2]:
def disp(expr):
    display(Math(vlatex(simplify(expr))))
    
def disp_eq(lhs,expr):
    display(Math("$${0} = {1}$$".format(lhs, vlatex(simplify(expr)))))

The python library sympy has been immensely helpful in sorting all of this out, and helping do the derivatives symbolically.  First, some symbols need to be set up.  For now, these are just symbols and won't have any values associated with them.  This is so the equations can be manipulated and we can see the results.

additionally $$\theta_0$$ and $$\theta_1$$ are set up as functions of time, and the initial derivatives wrt time are taken

In [3]:
t = symbols('t')
g = symbols('g')
l = symbols('l0:2')
m = symbols('m0:2')
r = symbols('r0:2')
I = symbols('I0:2')

theta = [w(t) for w in symbols('theta0:2')]
theta_dot = [Derivative(w, t) for w in theta]
theta_ddot = [Derivative(w, t, t) for w in theta]

## System State Equations

$$ \begin{align}
x_0=& r_{0} \cos{\left (\theta_0 \right )} & \\
y_0=& r_{0} \sin{\left (\theta_0 \right )} & \\
x_1=& l_{0} \cos{\left (\theta_0 \right )} + r_{1} \cos{\left (\theta_0 + \theta_1 \right )} & \\
y_1=& l_{0} \sin{\left (\theta_0 \right )} + r_{1} \sin{\left (\theta_0 + \theta_1 \right )} & \\
\end{align} 
$$

In [4]:
x = [None] * 2
y = [None] * 2
x_dot = [None] * 2
y_dot = [None] * 2

x[0] = r[0] * cos(theta[0])
y[0] = r[0] * sin(theta[0])
x[1] = l[1] * cos(theta[0]) + r[1] * cos(theta[0] + theta[1])
y[1] = l[1] * sin(theta[0]) + r[1] * sin(theta[0] + theta[1])

x_dot[0] = diff(x[0], t)
y_dot[0] = diff(y[0], t)
x_dot[1] = diff(x[1], t)
y_dot[1] = diff(y[1], t)

disp_eq("\dot{x_0}",x_dot[0])
disp_eq("\dot{y_0}",y_dot[0])
disp_eq("\dot{x_1}",x_dot[1])
disp_eq("\dot{y_1}",y_dot[1])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Kinetic Energy

$$
\begin{align}
K =& \tfrac{1}{2}I_0\omega_0^2 + \tfrac{1}{2}m_0v_0^2 + \tfrac{1}{2}I_1\omega_1^2 + \tfrac{1}{2}m_1v_1^2 & \\
\omega_0 =& \dot{\theta_0} & \\
\omega_1 =& \dot{\theta_0} + \dot{\theta_1} & \\
K =& \tfrac{1}{2}I_0\dot{\theta_0}^2 + \tfrac{1}{2}m_0v_0^2 + \tfrac{1}{2}I_1(\dot{\theta_0} + \dot{\theta_1})^2 + \tfrac{1}{2}m_1v_1^2 & \\
v =& \dot{x} + \dot{y} & \\
K =& \tfrac{1}{2}I_0\dot{\theta_0}^2 + \tfrac{1}{2}m_0(\dot{x_0}^2 + \dot{y_0}^2) + \tfrac{1}{2}I_1(\dot{\theta_0} + \dot{\theta_1})^2 + \tfrac{1}{2}m_1(\dot{x_1}^2 + \dot{y_1}^2) & \\
\end{align}
$$

We'll let sympy handle the last substitution to get the equation in terms of only $$\theta$$

In [5]:
kinetic = (m[0] * (x_dot[0] ** 2 + y_dot[0] ** 2)
           + m[1] * (x_dot[1] ** 2 + y_dot[1] ** 2)
           + I[0] * (theta_dot[0]) ** 2
           + I[1] * (theta_dot[0] + theta_dot[1]) ** 2) / 2

disp_eq("K",kinetic)

<IPython.core.display.Math object>

## Potential Energy

$$
\begin{align}
U =& m_0gy_0 + m_1gy_1 & \\
\end{align}
$$

In [6]:
potential = (m[0] * g * y[0]) + (m[1] * g * y[1])

disp_eq("U",potential)

<IPython.core.display.Math object>

## Lagrange

$$ L = K - U $$

In [7]:
lagrange = kinetic - potential

disp_eq("L",lagrange)

<IPython.core.display.Math object>

In [8]:
L = [None] * 2

L_0_0 = diff(lagrange, theta_dot[0], t)
L_0_1 = diff(lagrange, theta[0])
L[0] = L_0_0 - L_0_1

L_1_0 = diff(lagrange, theta_dot[1], t)
L_1_1 = diff(lagrange, theta[1])
L[1] = L_1_0 - L_1_1

disp_eq(r"\frac{d}{dt}\left(\frac{\partial{L}}{\partial{\dot{\theta_0}}}\right)",L_0_0)
disp_eq(r"\frac{\partial{L}}{\partial{\theta_0}}",L_0_1)
disp_eq(r"\frac{d}{dt}\left(\frac{\partial{L}}{\partial{\dot{\theta_0}}}\right)",L_1_0)
disp_eq(r"\frac{\partial{L}}{\partial{\theta_1}}",L_1_1)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [9]:
disp_eq(r"\frac{d}{dt}\left(\frac{\partial{L}}{\partial{\dot{\theta_0}}}\right)- \frac{\partial{L}}{\partial{\theta_0}}",L[0])
disp_eq(r"\frac{d}{dt}\left(\frac{\partial{L}}{\partial{\dot{\theta_0}}}\right)- \frac{\partial{L}}{\partial{\theta_1}}",L[1])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [10]:
solution = solve(L, theta_ddot)

In [11]:
disp_eq(r"\ddot{\theta_0}",solution[theta_ddot[0]])
disp_eq(r"\ddot{\theta_1}",solution[theta_ddot[1]])

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Now we can substitute numbers into our equation to perform the actual calculations.

In [12]:
g_new = S(9.8)
l_new = [S(1.0), S(1.0)]
m_new = [S(1.0), S(1.0)]
r_new = [temp_l/2 for temp_l in l_new]
I_new = [(temp_m * temp_l**2)/12 for temp_m,temp_l in zip(m_new,l_new)]

values = [(g,g_new),
          (l[0],l_new[0]),
          (l[1],l_new[1]),
          (m[0],m_new[0]),
          (m[1],m_new[1]),
          (r[0],r_new[0]),
          (r[1],r_new[1]),
          (I[0],I_new[0]),
          (I[1],I_new[1])]

In [13]:
disp_eq(r"\ddot{\theta_0}",solution[theta_ddot[0]].subs(values))
disp_eq(r"\ddot{\theta_1}",solution[theta_ddot[1]].subs(values))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [14]:
inputs = [(theta_dot[0], a), (theta[0], b), (theta_dot[1], c), (theta[1], d)]

LS = [None] * 2
theta0_ddot_function = lambdify((b, a, d, c), simplify(solution[theta_ddot[0]]).subs(values).subs(inputs))
theta1_ddot_function = lambdify((b, a, d, c), simplify(solution[theta_ddot[1]]).subs(values).subs(inputs))

In [15]:
def double_pendulum_deriv(this_state, time_step):
    this_theta0, this_theta0_dot, this_theta1, this_theta1_dot = this_state

    next_theta0_dot = this_theta0_dot
    next_theta1_dot = this_theta1_dot

    next_theta0_ddot = theta0_ddot_function(*this_state)
    next_theta1_ddot = theta1_ddot_function(*this_state)

    return np.array([next_theta0_dot, next_theta0_ddot, next_theta1_dot, next_theta1_ddot])

In [16]:
def init():
    line.set_data([], [])
    time_text.set_text('')
    return line, time_text


def animate(i):
    thisx = [0, x1_pos[i], x2_pos[i]]
    thisy = [0, y1_pos[i], y2_pos[i]]

    line.set_data(thisx, thisy)
    time_text.set_text(time_template % (i * dt))
    return line, time_text

In [17]:
dt = 0.05
t = np.arange(0.0, 15, dt)

theta0_initial = 0.0
theta0_dot_initial = 0.0
theta1_initial = 0.0
theta1_dot_initial = 0.0

initial_state = np.radians([theta0_initial, theta0_dot_initial, theta1_initial, theta1_dot_initial])

pos = integrate.odeint(double_pendulum_deriv, initial_state, t)

x1_pos = float(l_new[0]) * np.cos(pos[:, 0])
y1_pos = float(l_new[0]) * np.sin(pos[:, 0])

x2_pos = x1_pos + float(l_new[1]) * np.cos(pos[:, 0] + pos[:, 2])
y2_pos = y1_pos + float(l_new[1]) * np.sin(pos[:, 0] + pos[:, 2])

fig = plt.figure()
ax = fig.add_subplot(111, autoscale_on=False, xlim=(-2, 2), ylim=(-2, 2))
ax.grid()
ax.set_aspect('equal', adjustable='box')

line, = ax.plot([], [], 'k-', lw=4, solid_capstyle='round')
time_template = 'time = %.2fs'
time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)

ani = animation.FuncAnimation(fig, animate, frames=len(pos),
                              interval=25, blit=True, init_func=init)

In [18]:
HTML(ani.to_html5_video())